In [4]:
from numpy.linalg import inv,qr
from numpy import linalg as la
from scipy.linalg import lu, lu_solve, lu_factor,cholesky
from numpy.linalg import det
import numpy as np

In [3]:
""" --------------------GIẢI PHƯƠNG TRÌNH A.X = B VỚI PHÂN RÃ LU------------------------"""
## Hàm tạo ma trận hoán vị P từ pivot = linalg.lu_factor(A)[1]
def permutationMatrix(pivot):
    # Khởi tạo P bằng ma trận đơn vị
    P = np.identity(len(pivot))

    # i: index; r: element
    for i, r in enumerate(pivot):
        # pivot[i] = r
        # Hoán đổi dòng r và dòng pivot[i] của ma trận I
        I = np.identity(len(pivot))

        temp    = I[i, :].copy()
        I[i, :] = I[r, :]
        I[r, :] = temp

        P = P.dot(I)
    return P

# Kiểm tra định thức các ma trận con
def check_det(vector_heso):
    if vector_heso[0, 0] == 0:
        return False
    for i in range(2, vector_heso.shape[0] + 1):
        if det(vector_heso[:i, :i]) == 0:
            return False
    return True

# Giải hệ phương trình A*X = B 
def solve_LU_factorization(vector_heso, vector_heso_tudo):
    try:
        # Kiểm tra ma trận trước khi phân rã
        if vector_heso.shape[0] != vector_heso.shape[1]:
            raise ValueError('Ma trận hệ số không phải ma trận vuông')
                
        LU = lu_factor(vector_heso)
        # Ma trận tam giác trên
        U = np.triu(LU[0])
        # Ma trận tam giác dưới
        L = LU[0] - U + np.identity(len(vector_heso_tudo))

        if check_det(vector_heso):
            # Giải A = L*U, ma trận vuông
            x = lu_solve((LU),vector_heso_tudo)
            return x
        else:
            # Giải P.A = L.U, ma trận hình chữ nhật
            # Y = P.T *B *inv(L)
            Y = inv(L) @ vector_heso_tudo @ permutationMatrix(LU[1]).T
            x = Y @ inv(U)
            return x

    except ValueError as ve:
        return (f'ValueError: {ve}')
    except Exception as e:
        return (f'Có lỗi xảy ra: {e}')

In [5]:
""" --------------------GIẢI PHƯƠNG TRÌNH A.X = B VỚI PHÂN RÃ QR------------------------"""
""" A = Q x R
    Q: Ma trận trực chuẩn, để ma trận trực chuẩn phải trực giao
    R: Ma trận tam giác trên

    Kiểm tra ma trận Q có trực giao 
    + Q x (Q.T)  = (Q.T) x Q = I
    + Q x inv(Q) = I
    Nhân inv(Q) cho 2 vế của  (Q.T) x Q = I
    <=> inv(Q) x Q x Q.T = I.inv(Q) 
    <=> I x Q.T = I.inv(Q) (Q x inv(Q) = I)
    <=> Q.T = inv(Q)
    * => M trận trực giao khi Q.T = inv(Q) """

def is_orthogonal(Q):
    try:
        # Kiểm tra xem kích thước của ma trận Q có phải là vuông không
        if Q.shape[0] != Q.shape[1]:
            raise ValueError('Ma trận Q không phải ma trận vuông')
        else:
            if np.allclose(Q.T, inv(Q)):
                return True
            return False
    except ValueError as ve:
        return (f'ValueError: {ve}')
    except Exception as e:
        return (f'Có lỗi xảy ra: {e}')

def norm_each_cols(Q):
    for i in range(Q.shape[1]):
        Q_norm = np.linalg.norm(Q[:,i])
        if Q_norm != 0:
            print(Q_norm)
            return False
    return True

""" Phân rã QR: A = Q.R
    Hệ pt tuyến tính: A.X = B
    Ta có: Q.R.X = B
    Đặt R.X = Y
    <=> Q.Y = B
    => Y = B.inv(Q)
    => X = Y.inv(R)
    """

def solve_QR_factorization(vector_heso, vector_heso_tudo):
    try:
        # Kiểm tra ma trận trước khi phân rã
        if vector_heso.shape[0] != vector_heso.shape[1]:
            raise ValueError('Ma trận hệ số không phải ma trận vuông')
        
        Q,R = qr(vector_heso)
        # Kiểm tra ma trận Q và R có khả nghịch không?
        if det(Q)!= 0 and det(R)!= 0:
            Y = vector_heso_tudo @ inv(Q)
            X = np.linalg.solve(R, Y)
            return X
        else:
             raise ValueError('Ma trận Q hoặc R không khả nghịch')

    except ValueError as ve:
        return (f'ValueError: {ve}')
    except Exception as e:
        return (f'Có lỗi xảy ra: {e}')

In [ ]:
"""-------------------------- GIẢI PHƯƠNG TRÌNH A.X = B VỚI PHÂN RÃ CHOLESKY---------------------------------"""
# Để giải pt AX = B thì ma trận A phải là ma trận vuông đối xứng và xác định dương
# -------------------------- KIỂM TRA MA TRẬN ĐỐI XỨNG (Symmetric matrix)---------------------------------
# 1. Kiểm tra ma trận vuông
# 2. Kiểm tra đối xứng.
def is_SymmetricMatrix(A):
    try:
        if A.shape[0] != A.shape[1]: # 1
            raise ValueError('Ma trận không phải là ma trận vuông')
        for i in range(A.shape[0]):
            for j in range(i+1, A.shape[1]): #2
                if A[i][j] != A[j][i]:
                    return False  
        return True
    except ValueError as ve:
        return f'ValueError: {ve}'
    except Exception as e:
        return f'Có lỗi xảy ra: {e}'
    
# -------------------------- KIỂM TRA MA TRẬN XÁC ĐỊNH DƯƠNG (POSITIVE DEFINITE)---------------------------------
# Một ma trận bán xác định dương là một ma trận Hermitian mà tất cả các trị riêng của nó đều không âm.
def is_PositiveDefinite_Matrix(A):
    eigenValues, eigenVectors = la.eig(A)
    pos_def = np.all(eigenValues > 0)
    if (pos_def == False):
        return True
    return True

# -------------------------- HÀM TẠO MA TRẬN XÁC ĐỊNH DƯƠNG (POSITIVE DEFINITE)---------------------------------
def create_matrix_positive_definite(m, n, start, end):
    A       = None
    pos_def = False

    while (pos_def == False):
        A = np.random.randint(start, end, (m, n))
        for i in range(A.shape[0]):
            for j in range(i):
                A[j][i] = A[i][j]
        test    = np.linalg.eigvalsh(A)
        pos_def = np.all(test > 0)
    return A

# -------------------------- HÀM GIẢI PHƯƠNG TRÌNH---------------------------------
def solve_Cholesky_factorization(A,B):
    try:
        if is_SymmetricMatrix(A) == False or is_PositiveDefinite_Matrix(A) == False:
            raise ValueError('Ma trận hệ số không đủ điều kiện để phân rã Cholesky')
        """ A.X = B
            Phân rã Cholesky: A = L x (L.T) hoặc A = (U.T) x U
            => (U.T) x U x X = B
            Đặt U x X = Y
            => (U.T) x Y = B <=> Y = B x inv(U.T)
            Giải X = Y x inv(U)
            """
        U = cholesky(A, lower = False)
        Y = B @ inv(U.T)
        X = np.linalg.solve(Y, inv(U))
        return X
    except ValueError as ve:
        return (f'ValueError: {ve}')
    except Exception as e:
        return (f'Có lỗi xảy ra: {e}')
